<center><h1>Mini Project 3</h1></center>
<br>
<center><font size="5">Name - Spandan Patil</font></center>
<br>
<center><font size="4">AReM Data</font></center>

## 1. Time Series Classification Part 1: Feature Creation/Extraction

### (a) Download Data

Package imports

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict
from pathlib import Path
import bootstrapped.bootstrap as bs
import bootstrapped.stats_functions as bs_stats
from scipy.stats import skew

Get the AReM Data Set

In [33]:
# Getting a list of all classes for which we are having datasets.
path = Path("../data/AReM")
categories = [f.name for f in path.iterdir() if f.is_dir()]

print(categories)

['bending1', 'bending2', 'cycling', 'lying', 'sitting', 'standing', 'walking']


### (b) Test and Train Data

In [34]:
# Creating a dictionary of cls having a list containing path to each of the training dataset.
train_set = defaultdict(list)

for cls in categories:
    if cls == 'bending1':
        for i in range(3, 8): 
            train_set[cls].append(f"../data/AReM/{cls}/dataset{i}.csv")
    elif cls == 'bending2':
        for i in range(3, 7): 
            train_set[cls].append(f"../data/AReM/{cls}/dataset{i}.csv")
    else:
        for i in range(4, 16):
            train_set[cls].append(f"../data/AReM/{cls}/dataset{i}.csv")

In [35]:
# Creating a dictionary of cls having a list containing path to each of the testing dataset.
test_set = defaultdict(list)

for cls in categories:
    if cls == 'bending1' or cls == 'bending2':
        for i in range(1, 3): 
            test_set[cls].append(f"../data/AReM/{cls}/dataset{i}.csv")
    else:
        for i in range(1, 4):
            test_set[cls].append(f"../data/AReM/{cls}/dataset{i}.csv")

### (c) Feature Extraction

#### i. Research

The following Time-domain features are commonly used in time series classification include:

<ul>
    <li>Minimum: The lowest value in the time series.</li>
    <li>Maximum: The highest value in the time series.</li>
    <li>Mean: The average value of the time series.</li>
    <li>Median: The middle value when the time series is ordered.</li>
    <li>Standard deviation: A measure of the spread of values in the time series.</li>
    <li>First quartile: The 25th percentile of the ordered time series.</li>
    <li>Third quartile: The 75th percentile of the ordered time series.</li>
    <li>Variance: A measure of variability in the time series.</li>
    <li>Higher-order moments: Statistical measures beyond mean and variance.</li>
    <li>Coefficients of an AR (Autoregressive) model: Parameters that capture the dynamic behavior of the time series.</li>
    <li>Energy in specific frequency intervals: Obtained from Discrete Fourier Transform (DFT) or Discrete Wavelet Transform (DWT).</li>
</ul>

src - https://stats.stackexchange.com/questions/50807/features-for-time-series-classification

#### ii. Extraction

In [36]:
# Creating a list of all the columns which will be present in our new dataset having the time domain features.
cols = ['Instance']
for i in range(1, 7):
    cols.extend([f'min{i}', f'max{i}', f'mean{i}', f'median{i}', f'std{i}', f'1st_quart{i}', f'3rd_quart{i}'])

# Creating an empty dataframe with the list of columns.
extracted_df = pd.DataFrame(columns=cols)

# Creating a list of columns present in our dataset, to keep with reading the csv files.
inst_cols = ["time", "avg_rss12", "var_rss12", "avg_rss13", "var_rss13", "avg_rss23", "var_rss23"]

In [37]:
# Reading all the training datasets.

# For each of the classes in our training dictionary
for cls in train_set:
    # We get the index and path for that dataset.
    for idx, inst_path in enumerate(train_set[cls]):
        # Here we are handling the special case for bending2/dataset4.csv by using space as a separator instead of comma.
        if cls == "bending2" and idx == 1:
            # Reading the csv into a dataframe.
            df = pd.read_csv(inst_path, sep=" ", skiprows=5, header=None, names=inst_cols, usecols=range(7))
        else:
            df = pd.read_csv(inst_path, skiprows=5, header=None, names=inst_cols, usecols=range(7))
        # Here we are creating a new row which will be appended to our dataframe having all the time domain features we require from this dataset.
        new_row = []
        new_row.append(len(extracted_df)+1)
        # We are considering only the 6 time series data, ignoring the time column.
        for i in range(1, 7):
            new_row.extend([np.min(df[df.columns[i]]), np.max(df[df.columns[i]]), np.mean(df[df.columns[i]]), np.median(df[df.columns[i]]), np.std(df[df.columns[i]]), np.percentile(df[df.columns[i]], 25), np.percentile(df[df.columns[i]], 75)])
        extracted_df.loc[len(extracted_df)] = new_row        

In [38]:
# Reading all the testing datasets.

# For each of the classes in our testing dictionary
for cls in test_set:
    # We get the index and path for that dataset.
    for idx, inst_path in enumerate(test_set[cls]):
        # Reading the csv into a dataframe.
        df = pd.read_csv(inst_path, skiprows=5, header=None, names=inst_cols, usecols=range(7))
         # Here we are creating a new row which will be appended to our dataframe having all the time domain features we require from this dataset.
        new_row = []
        new_row.append(len(extracted_df)+1)
        # We are considering only the 6 time series data, ignoring the time column.
        for i in range(1, 7):
            new_row.extend([np.min(df[df.columns[i]]), np.max(df[df.columns[i]]), np.mean(df[df.columns[i]]), np.median(df[df.columns[i]]), np.std(df[df.columns[i]]), np.percentile(df[df.columns[i]], 25), np.percentile(df[df.columns[i]], 75)])
        extracted_df.loc[len(extracted_df)] = new_row        

In [ ]:
# As we can see we have our new dataset having time domain features for each of the 6 time series.
extracted_df.head()

,Instance,min1,max1,mean1,median1,std1,1st_quart1,3rd_quart1,min2,max2,...,std5,1st_quart5,3rd_quart5,min6,max6,mean6,median6,std6,1st_quart6,3rd_quart6
0,1.0,35.00,47.40,43.954500,44.33,1.557210,43.00,45.00,0.0,1.70,...,1.997520,35.3625,36.50,0.0,1.79,0.493292,0.43,0.512971,0.00,0.94
1,2.0,33.00,47.75,42.179812,43.50,3.666840,39.15,45.00,0.0,3.00,...,3.845436,30.4575,36.33,0.0,2.18,0.613521,0.50,0.523771,0.00,1.00
2,3.0,33.00,45.75,41.678063,41.75,2.241152,41.33,42.75,0.0,2.83,...,2.408514,28.4575,31.25,0.0,1.79,0.383292,0.43,0.388759,0.00,0.50
3,4.0,37.00,48.00,43.454958,43.25,1.384653,42.50,45.00,0.0,1.58,...,2.486268,22.2500,24.00,0.0,5.26,0.679646,0.50,0.621885,0.43,0.87
4,5.0,36.25,48.00,43.969125,44.50,1.616677,43.31,44.67,0.0,1.50,...,3.314843,20.5000,23.75,0.0,2.96,0.555312,0.49,0.487318,0.00,0.83


In [40]:
extracted_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 88 entries, 0 to 87
Data columns (total 43 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Instance    88 non-null     float64
 1   min1        88 non-null     float64
 2   max1        88 non-null     float64
 3   mean1       88 non-null     float64
 4   median1     88 non-null     float64
 5   std1        88 non-null     float64
 6   1st_quart1  88 non-null     float64
 7   3rd_quart1  88 non-null     float64
 8   min2        88 non-null     float64
 9   max2        88 non-null     float64
 10  mean2       88 non-null     float64
 11  median2     88 non-null     float64
 12  std2        88 non-null     float64
 13  1st_quart2  88 non-null     float64
 14  3rd_quart2  88 non-null     float64
 15  min3        88 non-null     float64
 16  max3        88 non-null     float64
 17  mean3       88 non-null     float64
 18  median3     88 non-null     float64
 19  std3        88 non-null     float64


#### iii. Standard Deviation

In [45]:
# Here is the function to calculate the confidence interval for each of the time domain features.
def cal_ci(data, ci=90):
    data = np.array(data).flatten()
    res = bs.bootstrap(data, stat_func=bs_stats.std, alpha=(100-ci)/100)
    return res.lower_bound, res.upper_bound, res.value

# Here we are computing the confidence intervals for each of the time domain feature
ci_intervals = {col: cal_ci(extracted_df[col]) for col in extracted_df.columns[1:]}

# Creating a dataframe to display our results.
df_ci = pd.DataFrame(ci_intervals, index=['Lower Bound', 'Upper Bound', 'Standard Deviation']).T
df_ci["interval_width"] = df_ci["Upper Bound"] - df_ci["Lower Bound"]
df_ci

,Lower Bound,Upper Bound,Standard Deviation,interval_width
min1,8.308240,10.811091,9.515445,2.502851
max1,3.513822,5.444607,4.369322,1.930785
mean1,4.757019,5.933797,5.305314,1.176778
median1,4.847203,6.059964,5.409056,1.212761
std1,1.587030,1.956356,1.760219,0.369326
1st_quart1,5.629197,6.707413,6.118526,1.078216
3rd_quart1,4.413105,5.903851,5.109643,1.490746
min2,0.000000,0.000000,0.000000,0.000000
max2,4.686502,5.469686,5.033882,0.783184
mean2,1.434065,1.744888,1.565194,0.310823


#### iv. Select Features

In [ ]:
# Here we are checking if each of the time domain features are rightly or leftly skewed.
skw_val = extracted_df.apply(skew)
count_p = 0
count_n = 0
for val in skw_val:
    if val > 0:
        count_p += 1
    elif val < 0:
        count_n += 1

print(count_p, count_n)

29 11


Based on my judgement i will select Mean, Standard Deviation and 3rd Quartile as the three most important time domain features. The reasoning is as follows:

<ul>
    <li> Mean : This tell us the average of the time series distribution.
    <li> Standard Deviation : This will tell us the variability and spread of the data points around the mean.
    <li> 3rd Quartile : This will tell us about the right-skewness of the distribution. We take 3rd Quartile as compared to 1st Quartile as we can see above most of our features are right skewed.
</ul>

## 2. ISLR 3.7.4

### (a) Linear Train

The RSS will be lower for cubic regression compared to linear regression for the training data, because even though the true relation is linear, the cubic model will be more flexible, so it can fit the training data more closely thus capturing noise and random fluctuations, resulting in lower RSS.

### (b) Linear Test

The RSS will be lower for the linear regression compared to cubic regression for the testing data, because, the true relation is linear and due to overfitting the cubic model will have captured some noise thus resulting in higher RSS.

### (c) Not Linear Train

The RSS will be lower for the cubic regression compared to the linear regression, because the we know that the true relation is non-linear, thus the cubic model will be able to capture the non-linearity of the data better compared to the linear model, thus resulting in lower RSS.

### (d) Not Linear Testing

There is not enough information to predict the RSS. Since, we dont know the degree of non-linearity of the true relation, the RSS can be lower for cubic model, if the degree of non-linerity is sufficiently higher, but otherwise the RSS will be higher since the cubic model can overfit the training data.

## 3. ISLR 3.7.3 - Extra Practice 

a) option iii. (For a fixed value of IQ and GPA, males earn more on average than females provided that the GPA is high enough) is correct, since if we input the value of x3 in the equation and find the difference between that of female and male we get the equation -> Sal(Female) - Sal(Male) = 35 - 10 * GPA. Hence, we can observe that for lower value of GPA, Female will have a higher salary on avg compared to male, but for higher values of GPA, Male will have a higher salary on avg compared to female.

b) Sal(Female) = 50 + 20 * (4) + 0.07 * (110) + 35 * (1) + 0.01 * (4.0) * (110) − 10 * (4.0) * (1) = 137.1. Thus the predicted Salary of female will be 137,100 USD.

c) False, Since we can determine the impact of a term based only on the magnitude of their coefficient, we would need check the p-value of the term against the confidence interval to determine its statistical significances.
​


## 3. ISLR 3.7.5 - Extra Practice 